In [ ]:
pip install geopandas

     |████████████████████████████████| 972kB 2.4MB/s 
     |████████████████████████████████| 10.9MB 13.5MB/s 
     |████████████████████████████████| 14.8MB 328kB/s 


In [ ]:
# load packages
from math import sin, cos, sqrt, atan2, radians
import folium
from folium import FeatureGroup, LayerControl
from folium.plugins import TimestampedGeoJson
from folium import plugins
import json
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.io import gbq
from google.cloud import bigquery
import geopandas as gpd
from shapely.geometry import Polygon
from shapely import wkt
import re
import geopy.distance
%matplotlib inline
from google.colab import drive # to allow colab save file in my drive
drive.mount('/content/drive')
from google.colab import auth
from google.colab import files
auth.authenticate_user()
print('Authenticated')
cwd=os.getcwd()
cwd
%unload_ext google.colab.data_table

Mounted at /content/drive
Authenticated
The google.colab.data_table extension is not loaded.


In [ ]:
# define project id
project_id = "fulfillment-dwh-production"
client = bigquery.Client(project = project_id)

### Get Info for the Vendor
##### Please update your desired entity ID and vendor code in the below code

In [ ]:
query1 = """

select distinct vendor_code
, v.name
, h.settings.delivery_fee.amount DF
, TO_JSON_STRING(v.location) as location
, TO_JSON_STRING(h.shape) as shape
, h.active_from
from cl.vendors_v2 v
left join unnest(delivery_areas) de
left join unnest(history) h
left join unnest(porygon) p
where entity_id = "TB_AE" and h.active_to is Null and de.is_deleted = False 
and vendor_code = '603414'
order by 3, 6 desc

"""

bigquery1 = client.query(query1).to_dataframe()
bigquery1.head(50)

,vendor_code,name,DF,location,shape,active_from
0,603414,"WEST WING, Al Barsha 3",0.0,"""POINT(55.2075347 25.0803116)""","""POLYGON((55.2196884155273 25.1097720137809, 5...",2020-10-04 07:30:54.511780+00:00
1,603414,"WEST WING, Al Barsha 3",0.0,"""POINT(55.2075347 25.0803116)""","""POLYGON((55.2196884155273 25.1097720137809, 5...",2020-08-26 19:38:08.993279+00:00
2,603414,"WEST WING, Al Barsha 3",0.0,"""POINT(55.2075347 25.0803116)""","""POLYGON((55.2519607334398 25.0473784184849, 5...",2020-07-16 14:29:49.702558+00:00
3,603414,"WEST WING, Al Barsha 3",3.0,"""POINT(55.2075347 25.0803116)""","""POLYGON((55.2270269393921 25.033116704347, 55...",2020-10-04 07:30:44.665441+00:00
4,603414,"WEST WING, Al Barsha 3",3.0,"""POINT(55.2075347 25.0803116)""","""POLYGON((55.2340221405029 25.1024661735572, 5...",2020-10-04 07:30:23.154984+00:00
5,603414,"WEST WING, Al Barsha 3",5.0,"""POINT(55.2075347 25.0803116)""","""POLYGON((55.2489394973964 25.0573256553693, 5...",2020-10-04 07:30:31.829529+00:00
6,603414,"WEST WING, Al Barsha 3",5.0,"""POINT(55.2075347 25.0803116)""","""POLYGON((55.2340221405029 25.1024661735572, 5...",2020-08-26 19:40:23.953177+00:00


In [ ]:
bigquery1=bigquery1.groupby('DF').first().reset_index()
bigquery1

,DF,vendor_code,name,location,shape,active_from
0,0.0,603414,"WEST WING, Al Barsha 3","""POINT(55.2075347 25.0803116)""","""POLYGON((55.2196884155273 25.1097720137809, 5...",2020-10-04 07:30:54.511780+00:00
1,3.0,603414,"WEST WING, Al Barsha 3","""POINT(55.2075347 25.0803116)""","""POLYGON((55.2270269393921 25.033116704347, 55...",2020-10-04 07:30:44.665441+00:00
2,5.0,603414,"WEST WING, Al Barsha 3","""POINT(55.2075347 25.0803116)""","""POLYGON((55.2489394973964 25.0573256553693, 5...",2020-10-04 07:30:31.829529+00:00


In [ ]:
query2 = """

select distinct dpc.vendor_code
, dpc.scheme_id
, dpc.travel_time_fee_config.travel_time_threshold
, dpc.travel_time_fee_config.fee
from cl.dynamic_pricing_config dpc 
where dpc.entity_id = "TB_AE" 
and dpc.vendor_code = '603414'
order by 4

"""

bigquery2 = client.query(query2).to_dataframe()
bigquery2.head(50)

,vendor_code,scheme_id,travel_time_threshold,fee
0,603414,637,10.433333,0
1,603414,637,15.75,3
2,603414,637,None,5


In [ ]:
query3 = """

SELECT * FROM `fulfillment-dwh-production.dl.tes_driving_time_formula`
where country_code = 'ae' and created_date >= '2020-10-02' and city_id is null
order by created_date desc limit 1
"""

bigquery3 = client.query(query3).to_dataframe()
bigquery3.head(50)

,id,country_code,city_id,formula,created,creator_id,created_at,created_by,updated_at,updated_by,region,merge_layer_run_from,created_date,merge_layer_created_at,merge_layer_updated_at
0,3f5b0f01-0a43-4c95-94c2-f6b085febfe3,ae,None,0.138x,2020-10-07 03:36:45.875811+00:00,system,2020-10-07 03:36:45.908424+00:00,system,2020-10-07 03:36:45.908424+00:00,system,eu,2020-10-07 03:00:00+00:00,2020-10-07,2020-10-07 07:18:18.976000+00:00,2020-10-07 07:18:18.976000+00:00


In [ ]:
bigquery2['coef'] = bigquery3['formula'].str[0:4].astype('float')
form = bigquery2.fillna(method='ffill').head(2)
form["distance"] = form["travel_time_threshold"].astype('float')/form["coef"] *60
form

,vendor_code,scheme_id,travel_time_threshold,fee,coef,distance
0,603414,637,10.433333,0,0.13,4815.384462
1,603414,637,15.75,3,0.13,7269.230769


### Extract vendor location and put it on the map

In [ ]:
loc1=bigquery1.iloc[0]["location"]
loc2=re.split(r'[,\s]', loc1) 
vend_y=float(loc2[0][7:])
vend_x=float(loc2[-1][:-3])

In [ ]:
m = folium.Map(location=[vend_x,vend_y],zoom_start=13)
tooltip = bigquery1.iloc[0]["name"]
folium.Marker([vend_x,vend_y], popup='<i>Mt. Hood Meadows</i>', tooltip=tooltip).add_to(m)
m

### Extract vendor delivery areas and plot them on the map

In [ ]:
da11=bigquery1.iloc[0]["shape"]
da12=re.split(r'[,\s]', da11)
da12[0]=da12[0][10:]
da12[-1]=da12[-1][:-3]
da_1_x = []
da_1_y = []
for i in range(0,len(da12),3):
  da_1_x.append(float(da12[i+1]))
  da_1_y.append(float(da12[i]))

da21=bigquery1.iloc[1]["shape"]
da22=re.split(r'[,\s]', da21)
da22[0]=da22[0][10:]
da22[-1]=da22[-1][:-3]
da_2_x = []
da_2_y = []
for i in range(0,len(da22),3):
  da_2_x.append(float(da22[i+1]))
  da_2_y.append(float(da22[i]))

da31=bigquery1.iloc[2]["shape"]
da32=re.split(r'[,\s]', da31)
da32[0]=da32[0][10:]
da32[-1]=da32[-1][:-3]
da_3_x = []
da_3_y = []
for i in range(0,len(da32),3):
  da_3_x.append(float(da32[i+1]))
  da_3_y.append(float(da32[i]))

In [ ]:
polygon_geom_7 = Polygon(zip(da_3_y, da_3_x))
polygon_geom_5 = Polygon(zip(da_2_y, da_2_x))
polygon_geom_0 = Polygon(zip(da_1_y, da_1_x))
crs = {'init': 'epsg:4326'}
 
polygon_7 = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom_7]) 
polygon_5 = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom_5])  
polygon_0 = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom_0])      

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
style2 = {'fillColor': '#00FFFFFF', 'lineColor': '#00FFFFFF'}
style3 = {'fillColor': '#178', 'lineColor': '#178'}
style4 = {'fillColor': '#1', 'lineColor': '#1'}

folium.GeoJson(polygon_7,style_function=lambda x:style2).add_to(m)
folium.GeoJson(polygon_5,style_function=lambda x:style3).add_to(m)
folium.GeoJson(polygon_0,style_function=lambda x:style4).add_to(m)

In [ ]:
m

### Calculate radius for each tier and draw the DPS circles
##### Please decide which option do you go for and only run the code in that part
##### Option A: Radius of each tier equals to the max distance between vendor and the border of each tier delivery area
##### Option B: Radius of each tier equals to the threshold of the travel time in its zone
##### Option C: Input radius according to your business knowlegde

##### Option A: Radius of each tier equals to the max distance between vendor and the border of each tier delivery area

In [ ]:
coords_1=(vend_x,vend_y)

pairs1 = zip(da_1_x, da_1_y)
distance1 = []
for point in pairs1:
  coords_2=(point[0],point[1])
  distance1.append(geopy.distance.vincenty(coords_1, coords_2).m)
radius1 = max(distance1)

pairs2 = zip(da_2_x, da_2_y)
distance2 = []
for point in pairs2:
  coords_2=(point[0],point[1])
  distance2.append(geopy.distance.vincenty(coords_1, coords_2).m)
radius2 = max(distance2)

pairs3 = zip(da_3_x, da_3_y)
distance3 = []
for point in pairs3:
  coords_2=(point[0],point[1])
  distance3.append(geopy.distance.vincenty(coords_1, coords_2).m)
radius3 = max(distance3)

##### Option B: Radius of each tier equals to the threshold of the travel time in its zone

In [ ]:
coords_1=(vend_x,vend_y)
radius1 = form.iloc[0]["distance"]
radius2 = form.iloc[0]["distance"]
pairs3 = zip(da_3_x, da_3_y)
distance3 = []
for point in pairs3:
  coords_2=(point[0],point[1])
  distance3.append(geopy.distance.vincenty(coords_1, coords_2).m)
radius3 = max(distance3)

##### Option C: Input radius according to your business knowlegde

In [ ]:
radius1 = input("Input radius1 in meter: ")
radius2 = input("Input radius2 in meter: ")
radius3 = input("Input radius3 in meter: ")

Input radius1 in meter: 3500
Input radius2 in meter: 5500
Input radius3 in meter: 8000


# ----------------------------------------------------------------------------
##### Selection stage ends here and run the following codes

In [ ]:
# Please update DPS distance thresholds here
folium.Circle(location=[vend_x,vend_y], popup='tier 1', radius=radius1, weight=1, color="red", fill=False).add_to(m)
folium.Circle(location=[vend_x,vend_y], popup='tier 2', radius=radius2, weight=1, color="red", fill=False).add_to(m)
folium.Circle(location=[vend_x,vend_y], popup='tier 3', radius=radius3, weight=1, color="red", fill=False).add_to(m)

In [ ]:
m